<a href="https://colab.research.google.com/github/NiharikaGanji07/Niharika_INFO5731_Spring2021/blob/main/In_class_exercise_08_1/In_class_exercise_08_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The eighth in-class-exercise (20 points in total, 3/30/2021)**

The data for this exercise is from the dataset you created from assignment three. Please perform answer the following questions based on your data:

## (1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.

In [ ]:
# Write your code here
!pip install polyglot

In [ ]:
!pip install pyicu

In [ ]:
!pip install morfessor

In [ ]:
!pip install pycld2

In [ ]:
%%bash
polyglot download sentiment2.en

[polyglot_data] Downloading package sentiment2.en to
[polyglot_data]     /root/polyglot_data...
[polyglot_data]   Package sentiment2.en is already up-to-date!


In [ ]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /root/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Package biocreative_ppi is already up-to-date!
       | Downloading package brown to /root/nltk_data...
       |   Package brown is already up-to-date!
       | Downloading package brown_tei to /root/nltk_data...
       |   Package brown_tei is already up-to-date!
       | Downloading package cess_cat to /root/nltk_data...
       |   Package cess_cat is already up-to-date!
       | Down

True

In [ ]:

nltk.download('stopwords')
Data = pd.read_csv('https://raw.githubusercontent.com/NiharikaGanji07/Niharika_INFO5731_Spring2021/main/reviwes.csv')
Data = Data.loc[:, ~Data.columns.str.contains('^Unnamed')]



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from nltk.corpus import stopwords
from textblob import TextBlob
stop = stopwords.words('english')
#lowercase
Data['Review'] = Data['Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#Removing Punctuation
Data['Review'] = Data['Review'].str.replace('[^\w\s]','')
#special charchters removal
import re
Data['Review'] = Data['Review'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
#stopwords removal
Data['Review'] = Data['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#tokenization
Data['Review'] = Data['Review'].apply(lambda x: str(TextBlob(x).correct()))
Data



,Title,Review,Clean_text,Annotation
0,"Every once in a while a movie comes, that trul...",every movie comes truly makes impact joaquins ...,"every once in a while a movie comes, that trul...",Positive
1,This is a movie that only those who have felt ...,movie felt alone isolated truly relate underst...,this is a movie that only those who have felt ...,Positive
2,"Truly a masterpiece, The Best Hollywood film o...",truly masterpiece best hollywood film 2019 one...,"truly a masterpiece, the best hollywood film o...",Positive
3,Joaquin Phoenix gives a tour de force performa...,joaquin phoenix gives tour de force performanc...,joaquin phoenix gives a tour de force performa...,Positive
4,Most of the time movies are anticipated like t...,time moves anticipated like end falling short ...,most of the time movies are anticipated like t...,Positive
...,...,...,...,...
101,"One of the worst movies I have ever seen. Ok, ...",one worst moves ever seen ok maybe hope surrou...,"one of the worst movies i have ever seen. ok, ...",Neutral
102,I didnt really know much about this movie. I h...,didn really know much movie hadn seen trailer ...,i didnt really know much about this movie. i h...,Neutral
103,Film was directed and produced poorly. Opportu...,film directed produced poorly opportunity take...,film was directed and produced poorly. opportu...,Negative
104,"Wow, what a movie! I have to admit, When I fir...",now movie admit first heard joaquin phoenix go...,"wow, what a movie! i have to admit, when i fir...",Positive


In [ ]:
from polyglot.text import Text
extract_sentiment = []
for i in Data['Review']:
  word = Text(i)
  for w in word.words:
    if w.polarity != 0:
      extract_sentiment.append(w)
extract_sentiment

from collections import Counter
word_counter= Counter(extract_sentiment)
count = pd.DataFrame(list(word_counter.most_common()), columns=['word', 'word_count'])
count.index = list(range(1, len(word_counter.most_common())+1))
count

,word,word_count
1,joke,147
2,like,54
3,good,54
4,best,30
5,great,28
...,...,...
507,advantage,1
508,disappointing,1
509,creep,1
510,pleasantly,1


## (2) (10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers. 

Reference code: https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [ ]:
# Write your code here
def function_encode(sentiment):
  s = 0
  if sentiment == 'Positive':
    s = 1
  elif sentiment == 'Negative':
    s = 2
  return s
TrueSentiment = []
for sentiment in Data['Annotation']:
  TrueSentiment.append(function_encode(sentiment))

#TextBlob
from textblob import TextBlob
TextBlobSentiment = []
for i in Data['Review']:
  polarity = TextBlob(i).sentiment.polarity
  if polarity > 0:
    TextBlobSentiment.append(1)
  elif polarity < 0:
    TextBlobSentiment.append(2)
  elif polarity == 0.0:
    TextBlobSentiment.append(0)
Data1 = pd.DataFrame(list(zip(df['Review'], TrueSentiment, TextBlobSentiment)), columns = ['Review', 'ActualValue', 'TextBlobPredicted'])
Data1



,Review,ActualValue,TextBlobPredicted
0,"Every once in a while a movie comes, that trul...",1,2
1,This is a movie that only those who have felt ...,1,0
2,"Truly a masterpiece, The Best Hollywood film o...",1,1
3,Joaquin Phoenix gives a tour de force performa...,1,2
4,Most of the time movies are anticipated like t...,1,1
...,...,...,...
101,NaN,0,1
102,NaN,0,1
103,Ratings for acting are overrated. This movie i...,2,2
104,NaN,1,1


In [ ]:
#TextBlob Acurracy
from sklearn.metrics import f1_score, accuracy_score
accuracy= accuracy_score(Data1['ActualValue'], Data1['TextBlobPredicted'])*100
F1score= f1_score(Data1['ActualValue'], Data1['TextBlobPredicted'], average = 'macro')
print("Text Blob Accuracy is",accuracy,"and F1 Score is",F1score)


Text Blob Accuracy is 50.943396226415096 and F1 Score is 0.4185191523345027


In [ ]:
#VADER
import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()
VaderSentiment = []
for i in Data['Review']:
  j = vader.polarity_scores(i)['compound']
  if j > 0:
    VaderSentiment.append(1)
  elif j < 0:
    VaderSentiment.append(2)
  elif j == 0.0:
    VaderSentiment.append(0)
Data2 = pd.DataFrame(list(zip(Data['Review'], TrueSentiment, VaderSentiment)), columns = ['Review', 'ActualValue', 'VADERPredicted'])
Data2

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Review,ActualValue,VADERPredicted
0,every movie comes truly makes impact joaquins ...,1,1
1,movie felt alone isolated truly relate underst...,1,1
2,truly masterpiece best hollywood film 2019 one...,1,1
3,joaquin phoenix gives tour de force performanc...,1,1
4,time moves anticipated like end falling short ...,1,1
...,...,...,...
101,one worst moves ever seen ok maybe hope surrou...,0,1
102,didn really know much movie hadn seen trailer ...,0,1
103,film directed produced poorly opportunity take...,2,2
104,now movie admit first heard joaquin phoenix go...,1,1


In [ ]:
#VADER Acurracy
from sklearn.metrics import f1_score, accuracy_score
accuracy= accuracy_score(Data2['ActualValue'], Data2['VADERPredicted'])*100
F1score= f1_score(Data2['ActualValue'], Data2['VADERPredicted'], average = 'macro')
print("VADER Accuracy is",accuracy,"and F1 Score is",F1score)

VADER Accuracy is 49.056603773584904 and F1 Score is 0.3743169398907103


In [108]:
#TFIDF-based Support Vector Machine (SVM)
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

data3 = pd.read_csv("/content/sample_data/reviwes.csv")
t1, t2 = sklearn.model_selection.train_test_split(data3, train_size=0.6, test_size=0.1)
p = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=100, 
                                           learning_rate='optimal', tol=None))])

t3 = p.fit(t1['Review'], t1['Annotation'])
t2['predicted sentiment'] = t3.predict(t2['Review'])
print(t2[['Title', 'Annotation', 'predicted sentiment']].head(5))

accuracy = accuracy_score(t2['Annotation'], t2['predicted sentiment'])*100
F1score = f1_score(t2['Annotation'], t2['predicted sentiment'], average='macro')

print('TFIDF-based Support Vector Machine (SVM) accuracy is ', accuracy,'and F1 score is',F1score)


                                                Title  ... predicted sentiment
93  What can I say about it that hasn't already be...  ...             Neutral
89  This has to be the dumbest DC movie yet. Way t...  ...             Neutral
58  I do agree that Joaquin's performance was grea...  ...            Negative
13  I quit relying on critic reviews years ago... ...  ...            Negative
60  The ending is what matters in a movie. And thi...  ...            Negative

[5 rows x 3 columns]
TFIDF-based Support Vector Machine (SVM) accuracy is  54.54545454545454 and F1 score is 0.5373737373737374


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [ ]:
#your analysis here
"""
Sentiment analysis also known as opinion mining is used to determine whether the data is positive, neutral or negtaive. 
Sentiment analysis helps organisations in monitoring about their data and providing a feedback about their services.
There are different methods to perform sentiment analysis

TextBlob:TextBlob is a Python (2 and 3) library for processing textual data. It provides a simple API for diving into 
common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, 
classification, translation, and more. TextBlob supports complex analysis and operations on textual data.

VADER:
Valence Aware Dictionary for Sentiment Reasoning is one of the model which is used for sentiment analysis. 
In this model the analysis is performed on the text whether it is positive or negative and at the same time on the emtion of the text too.
This model is avaliable in NLTK package and is applicable to unlabeled text data.

SVM:
A support vector machine (SVM) is a supervised machine learning model that uses classification algorithms for two-group classification problems. 
After giving an SVM model sets of labeled training data for each category, they’re able to categorize new text.

All these models have different performance scores and these perfromances are measured based upon the accuracy and F1 score.

For the above data SVM is having the highest F1 score and next is TextBlob and the last is VADER.
So, it can be inferred that SVM is the best model to perform the sentiment analysis.

"""